# Classifiers

**有监督学习**
|       | probabilistic     | non-probabilistic  |
| -------- | -------- | -------- |
| classification | Discriminative \\ Generative | SVM、perceptron、NN不用softmax  |
| regression | - | - |

<font color=blue>分类问题如果线性可分，用perceptron model。但实践中基本都是线性不可分问题，此时就要用MLP(multi-layer perceptron)来求解。</font>

## I. 分类问题的求解框架
$$
input \rightarrow  feature\ extraction(representation) \rightarrow classifier \underset{optimization}{\rightarrow}  cost function
$$

## II. 解概率分类问题的最优分类器：Bayes classifer
bayes classifier不涉及$P(y|x)$的具体形式，所以对discriminative和generative classifier都适用。
### 2.1 贝叶斯分类器的定义
1. 设定：\
(1)分类问题：$x\in R^d,y\in \{1, ..., K\}$,classifier c(x)为映射：$R^d\rightarrow\{1, ..., K\}$ \
(2)概率分布：似然$P(x|y=k)$,先验$P(y=k)=\phi_k, {\textstyle \sum_{k=1}^{K}}\phi_k=1$ \
(3)定义classifier c在已知x取值条件下的错误率为：$R(c_x)=R(c(x))=P(C(x)\ne y)$ 

2. 定义Bayes Classifier：$c_B(x)=\underset{k\in \{1, ..., K\}}{argmax} P(y=k|x)$ \
<font color=red>**含义：给定x，bayes classifier是使后验概率$P(y=k|x)$最大的分类器。**</font> \
以二分类问题为例，取$y\in \{0, 1\}$,则
$$c_B(x)= \left\{\begin{matrix}
1  &if,p(y=1|x)\ge p(y=0|x)  \\
0  &if,p(y=1|x)<  p(y=0|x) 
\end{matrix}\right.$$

### 2.2 性质：如果知道feature的概率分布，那么贝叶斯分类器能最小化misclassification的概率。

<font color=green>**以二分类问题为例的证明：$c_B$的error rate$R(c_B)$是所有分类器中最小的**</font>

取$\eta(x)=P(y=1|x)$,则$$
\begin{align}
 R(c_x)&=P(C(x)\ne y)\\
&= \left\{\begin{matrix}
P(y=0|x)  & if, c(x)=1 \\
P(y=1|x)  & if, c(x)=0 
\end{matrix}\right.\\
&=\left\{\begin{matrix}
1-\eta(x)  &if,c(x) & = 1 \\
\eta(x)  &if,c(x) & = 0
\end{matrix}\right.  \\
\end{align}
$$

尽管$\eta(x)$未知，但是它是由分布给定的，所以$R(c_x)$有下界：
$$
minR(c_x)=min((1-\eta(x)), \eta(x))
$$
<img src="./pics/bayes_classifier_pic1.jpg" style="zoom:40%">
因此，只要分类器能处处取到$minR(c_x)$，该分类器就是最优分类器。

根据$c_B$的定义:$$
\begin{align}
c_B(x)
&= \left\{\begin{matrix}
1  &if,p(y=1|x)\ge p(y=0|x)  \\
0  &if,p(y=1|x)<  p(y=0|x) 
\end{matrix}\right.\\
&= \left\{\begin{matrix}
1  &if,\eta(x)\ge 1-\eta(x)  \\
0  &if,\eta(x)< 1-\eta(x)
\end{matrix}\right.  \\
\end{align}
$$

此时有：$$
\begin{align}
R(c_B)
&= \left\{\begin{matrix}
p(y=0|x)  &if,p(y=1|x)\ge p(y=0|x)  \\
p(y=1|x)  &if,p(y=1|x)<  p(y=0|x) 
\end{matrix}\right.\\
&= \left\{\begin{matrix}
1-\eta(x)  &if,1-\eta(x)\le \eta(x) \\
\eta(x)  &if,1-\eta(x)> \eta(x)
\end{matrix}\right.  \\
&=minR(c_x)
\end{align}
$$

### 2.3 实现方式
1. <font color=red>**bayes classifier为分类问题的处理提供了基本原则。但一般问题中，$p(y|x)$是未知的，所以无法直接使用该classifier。**</font>
2. <font color=blue>**有两类确定$p(y|x)$的方法：Discriminative和Generative model。**</font>

## III. Generative model和Discriminative model
<font color=green>**关键区别：**两种model在处理分类问题时的核心差异在于寻找$p(y|x)$时使用的方法不同。</font>
1. Generative model假设x和y都是随机变量，有联合分布$p(x,y)$。利用关系$p(y|x)=\frac{p(x,y)}{p(x)}=\frac{p(x|y;\theta)p(y;\varphi )}{p(x)}$对先验和似然建模求参数
2. Discriminative model假设x是固定的，只有y是随机变量，直接对$p(y|x;\theta)$的分布建模求参数

### 3.1 **<font color=orange>Generative model</font>**
#### 3.1.1 模型的基本框架
1. 模型训练时：用贝叶斯分析方式对$𝑃(𝑦|𝑋)$做分解，转化为对先验分布$P(y_i;φ)$和似然$P(X|y_i;θ)$建模，用MLE或者CE做目标函数，求解参数θ和$\varphi$。\
贝叶斯分析：\
$P(y_i|X) = \frac{P(X|y_i)*P(y_i)}{P(X)} = \frac{P(X|y_i)*P(y_i)}{Σ_{y_j\in{Y}} P(X|y_j)*P(y_j)}$ \
求解MLE：\
$\hat{θ}=\underset{θ,φ}{argmax}\ L(θ,φ) = \underset{θ,φ}{argmax}\ \frac{P(X|y_i;θ)*P(y_i;φ)}{Σ_{y_j\in{Y}} P(X|y_j;θ)*P(y_j;φ)}$
2. 模型推理时：同样利用贝叶斯关系对目标函数做分析，求解优化函数。 \
利用贝叶斯分析： \
$\hat{y}=\underset{y}{argmax}P(y|X) = \underset{y}{argmax}\frac{P(X|y)*P(y)}{P(X)}$ \
<font color=red>由于y的取值与P(X)无关</font>，因此有：\
$\hat{y}=\underset{y}{argmax}\ P(X|y;\hat{θ})*P(y;\hat{φ})=\underset{y}{argmax}\ P(X,y;\hat{θ},\hat{φ})$

#### 3.1.2 例1：GDA——从Generative model的视角理解logistic regression
1. <font color=blue>**本质：似然是Gaussion的2分类问题用Generative model得到的后验分布正好是Logistic**</font>
2. 设定：
3. 分析过程：详见cs229笔记page13

### 3.2 **<font color=orange>Discriminative model</font>**
#### 3.2.1 模型的基本框架
1. 模型训练时：直接对$P(y|X)$的参数建模，用MLE或者CE做目标函数，求解参数θ:
$\hat{θ}=\underset{θ}{argmax}\ L(θ) = \underset{θ}{argmax}\ \underset{i}{Σ}logP(y_{i}|X_{i};θ)$
2. 模型推理时：直接用估计的条件概率分布求概率最大的类型： \
$\hat{y}=\underset{y}{argmax}P(y|X;\hat{θ})$

#### 3.2.2 例1：LR——从Discriminative model的视角理解logistic regression
直接以logistc函数的形态作为$p(y|x)$的近似。

<font color=green>**以二分类问题为例的证明：取$\bar R(c)$为正确率的期望，即$\bar R(c)= 1-R(c)$，则$\bar R(c_B)=max\bar R(c)$**</font> 

1. 补充定义classifer c的错误率：$R(c)=E_{(x,y)}1\{c(x)\ne y\}$，即，用$R(c_x)$的期望值来衡量分类器的错误率。
如果用错误率的期望来衡量分类器的整体错误率，则：$$
\begin{align*}
R(c)
&=E_{(x,y)}1\{c(x)\ne y\} \\
&=E_xE_{y|x}1\{c(x)\ne y\}\\
&=E_x[ {\textstyle \sum_{y|x}}P(y|x)*1\{c(x)\ne y\} ]\\
&=E_x[ P(y=0|x)*1\{c(x)=1\} + P(y=1|x)*1\{c(x)=0\}]
\end{align*}
$$

取$\eta(x)=P(y=1|x)$,则：
$$
\begin{align*}
E[R(c)] 
&=E_x[ (1-\eta(x))*1\{c(x)=1\} + \eta(x)*1\{c(x)=0\}]\\
&=E_{x:c(x)=1}[1-\eta(x)]+E_{x:c(x)=0}[\eta(x)]
\end{align*}
$$